In [ ]:
# This is Main function.
# Extracting streaming data from Twitter, pre-processing, and loading into MySQL
import credentials # Import api/access_token keys from credentials.py
import settings # Import related setting constants from settings.py 

import re
import tweepy
import mysql.connector
from textblob import TextBlob
# Streaming With Tweepy 
# http://docs.tweepy.org/en/v3.4.0/streaming_how_to.html#streaming-with-tweepy


# Override tweepy.StreamListener to add logic to on_status
class MyStreamListener(tweepy.StreamListener):
    '''
    Tweets are known as “status updates”. So the Status class in tweepy has properties describing the tweet.
    https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object.html
    '''
    
    def on_status(self, status):
        '''
        Extract info from tweets
        '''
        
        if status.retweeted:
            # Avoid retweeted info, and only original tweets will be received
            return True
        # Extract attributes from each tweet
        id_str = status.id_str
        created_at = status.created_at
        text = deEmojify(status.text)    # Pre-processing the text  
        sentiment = TextBlob(text).sentiment
        polarity = sentiment.polarity
        subjectivity = sentiment.subjectivity
        
        user_created_at = status.user.created_at
        user_location = deEmojify(status.user.location)
        user_description = deEmojify(status.user.description)
        user_followers_count =status.user.followers_count
        longitude = None
        latitude = None
        if status.coordinates:
            longitude = status.coordinates['coordinates'][0]
            latitude = status.coordinates['coordinates'][1]
            
        retweet_count = status.retweet_count
        favorite_count = status.favorite_count
        
        print(status.text)
        print("Long: {}, Lati: {}".format(longitude, latitude))
        
        # Store all data in MySQL
        if mydb.is_connected():
            mycursor = mydb.cursor()
            sql = "INSERT INTO {} (id_str, created_at, text, polarity, subjectivity, user_created_at, user_location, user_description, user_followers_count, longitude, latitude, retweet_count, favorite_count) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)".format(settings.TABLE_NAME)
            val = (id_str, created_at, text, polarity, subjectivity, user_created_at, user_location, \
                user_description, user_followers_count, longitude, latitude, retweet_count, favorite_count)
            mycursor.execute(sql, val)
            mydb.commit()
            mycursor.close()
    
    
    def on_error(self, status_code):
        '''
        Since Twitter API has rate limits, stop srcraping data as it exceed to the thresold.
        '''
        if status_code == 420:
            # return False to disconnect the stream
            return False

#clean the tweetes        
def clean_tweet(self, tweet): 
    ''' 
    Use sumple regex statemnents to clean tweet text by removing links and special characters
    '''
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) \
                                |(\w+:\/\/\S+)", " ", tweet).split()) 
def deEmojify(text):
    '''
    Strip all non-ASCII characters to remove emoji characters
    '''
    if text:
        return text.encode('ascii', 'ignore').decode('ascii')
    else:
        return None
    
#MySql connection stablish
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    passwd="Wonderbiz@987",
    database="TwitterDB",
    charset = 'utf8'
)
if mydb.is_connected():
    '''
    Check if this table exits. If not, then create a new one.
    '''
    mycursor = mydb.cursor()
    mycursor.execute("""
        SELECT COUNT(*)
        FROM information_schema.tables
        WHERE table_name = '{0}'
        """.format(settings.TABLE_NAME))
    if mycursor.fetchone()[0] != 1:
        mycursor.execute("CREATE TABLE {} ({})".format(settings.TABLE_NAME, settings.TABLE_ATTRIBUTES))
        mydb.commit()
    mycursor.close()

#autanticated with tweeters
auth  = tweepy.OAuthHandler(credentials.Api_key, credentials.Api_secret_key)
auth.set_access_token(credentials.Access_Token, credentials.Access_Token_Secret)
api = tweepy.API(auth)

#connection close

myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = api.auth, listener = myStreamListener)
myStream.filter(languages=["en"], track = settings.TRACK_WORDS)
# Close the MySQL connection as it finished
# However, this won't be reached as the stream listener won't stop automatically
# Press STOP button to finish the process.
mydb.close()


RT @ygent_official: #강승윤(KANG SEUNG YOON) 1st FULL ALBUM [PAGE] 
Online Fan signing event
Notice has been uploaded. 

▶️https://t.co/CHREBr…
Long: None, Lati: None
RT @GrogsGamut: Beat appearing before journalists who might ask questions
Long: None, Lati: None
Facebook TM - helping the rich and powerful step on the necks of the disenfranchised since 2004 https://t.co/0oFOPLSlCq
Long: None, Lati: None
#MondayMotivation
Be Connected with us -

Facebook - https://t.co/JI1AXSpY2i
Instagram - https://t.co/A5fkRa9oYa
Yo… https://t.co/O5G0ECBksb
Long: None, Lati: None
@GrogsGamut I think it might be an opportune moment for genuine journalists to step up and expose the bullshit that… https://t.co/13Ke3TPBKC
Long: None, Lati: None
RT @blanketcrap: Oyster Picks and Oyster shells from excavated Dee r Mt Morgan 1500ft above sealevel, carved bone artifacts, Ocean was a da…
Long: None, Lati: None
RT @WildeOscarCat: #Mondaythoughts #Sweet SOKA needs a new best friend!  #NYC #cats #sweet #neutered htt

RT @GOT7Charts: .@JacksonWang852 is the most liked K-Pop artist on Facebook for two consecutive weeks.

@GOT7Official #GOT7 https://t.co/vF…
Long: None, Lati: None
RT @BamBamland502: 210412 [Facebook] BamBam Play The Sims update

BamBam will be live on Facebook streaming The Sims 4!

⏰ Apr 15 at 10PM K…
Long: None, Lati: None
RT @kyawhsanhlaing1: The military is now charging families 120,000 Myanmar kyat ($85) to retrieve the bodies of relatives who died Friday,…
Long: None, Lati: None
RT @ChartsJackson: . @JacksonWang852 was the #1 most liked Kpop artist on Facebook for a 2nd week in a row! https://t.co/eZjK4OekNT
Long: None, Lati: None
Hello guys! Don't forget to like 
Xiaomi Southwoods Mall fb page! 

https://t.co/JSL1Vw9m0e
Long: None, Lati: None
Good luck and a shout out to all the shops in Exeter and Devon 👍

#exeter #devon #shoplocal @Tweet4Devon
Long: None, Lati: None
RT @BamBamland502: 210412 [Facebook] BamBam Play The Sims update

BamBam will be live on Facebook streaming The

https://t.co/OYoTodjeCt https://t.co/pVOPQLY4Mc
Long: None, Lati: None
RT @falling1004yoon: [INSIDE SEVENTEEN] Hoshi Spider Mixtape Behind

https://t.co/OYoTodjeCt https://t.co/pVOPQLY4Mc
Long: None, Lati: None
RT @BamBamland502: 210412 [Facebook] BamBam Play The Sims update

BamBam will be live on Facebook streaming The Sims 4!

⏰ Apr 15 at 10PM K…
Long: None, Lati: None
RT @BamBamland502: 210412 [Facebook] BamBam Play The Sims update

BamBam will be live on Facebook streaming The Sims 4!

⏰ Apr 15 at 10PM K…
Long: None, Lati: None
RT @elusive_6788: 🤠 Cowboy romance readers, ask me anything about my story "Loving Jack" in "Cowboy Desire" on Sunday, April 18.
Where: htt…
Long: None, Lati: None
@kyualakyu_ hello im so sorry for what happened. I have this saved before, just in case I came across a scammer na… https://t.co/ElspZ7ff5j
Long: None, Lati: None
RT @ea991_botswana: The winners from our Week 1 Lucky Draw Competition have been posted on our Facebook Page! 

https://t.co/k0ymxCPZI

Hey, @DirtyCleanFood, you may want to have a word with @Facebook - just tried to share your page to my newsfeed and Facebook blocked it.
Long: None, Lati: None
RT @HuffPostTech: The heads of Facebook-owned WhatsApp and Messenger services told Barr and his U.K. and Australian counterparts that Faceb…
Long: None, Lati: None
Bleak. Get your house in order Facebook. https://t.co/grLstJaJp5
Long: None, Lati: None
RT @ygent_official: #강승윤(KANG SEUNG YOON) 1st FULL ALBUM [PAGE] 
Online Fan signing event
Notice has been uploaded. 

▶️https://t.co/CHREBr…
Long: None, Lati: None
RT @BamBamland502: 210412 [Facebook] BamBam Play The Sims update

BamBam will be live on Facebook streaming The Sims 4!

⏰ Apr 15 at 10PM K…
Long: None, Lati: None
RT @MathewsFiona: Trend for 'tree glue': marketed as environmentally-friendly way to kill moths and ants is terrible. This stuff is horribl…
Long: None, Lati: None
fbm20140201a - #SevenYearsAgo
#CNY #celebrations at #ParkWest, #Singapore.

More on: https://t.c

RT @MaiKhoiOi: @szhang_ds Thank you for calling out Facebook @szhang_ds. Everything you say reflects my own experience with Facebook in Vie…
Long: None, Lati: None
RT @BamBamland502: 210412 [Facebook] BamBam Play The Sims update

BamBam will be live on Facebook streaming The Sims 4!

⏰ Apr 15 at 10PM K…
Long: None, Lati: None
RT @LatteandLive: Social Media Content- where to find it, create it and share it. 

Less than an hour to go. Join us on zoom or tune into t…
Long: None, Lati: None
RT @BJP4India: PM Shri @narendramodi will address public meetings on 12th April 2021 in West Bengal.

Watch on

• https://t.co/vpP0MI6iTu
•…
Long: None, Lati: None
RT @MahjongSoul_EN: Mahjong Soul X Kakegurui xx Collab Info Reveal!

One winner will be drawn at random on Twitter/Facebook. Follow us and…
Long: None, Lati: None
RT @MikeCarlton01: When two prime ministers - one Labor, one Liberal - denounce the Murdochracy attack on our democracy, you begin to think…
Long: None, Lati: None
RT @shiela_senpai

...

RT @WORLDMUSICAWARD: Happy Birthday to #EXO's gorgeous and multitalented #SEHUN! 👨‍🎤🎸🎂🍾🎁🎉🎇☀️👑 #HappySehunDay #28thSpringWithSEHUN  #세훈 @wea…
Long: None, Lati: None
AAPL Stock - Apple Inc. (NASDAQ:AAPL), Facebook, Inc. (NASDAQ:FB) - Apple To Impose New App Privacy Notifications S… https://t.co/bEw984Z0yK
Long: None, Lati: None
RT @carolecadwalla: Deep dive here. It’s such an important story because it reveals just how much Facebook does not give a shit about the r…
Long: None, Lati: None
RT @numloq: The road to self love is often lonely. People will see someone trying to change and that will shake their belief system sebab d…
Long: None, Lati: None
RT @yvemoongallery: @maxinejijiBTY @maxinejijiesOFC What's Up Mga Ka-Madam/Serr

👉AUTHORIZED DISTRIBUTOR HERE
✨ You're from QUEZON, PROVINC…
Long: None, Lati: None
RT @BamBamland502: 210412 [Facebook] BamBam Play The Sims update

BamBam will be live on Facebook streaming The Sims 4!

⏰ Apr 15 at 10PM K…
Long: None, Lati: None
RT @hoshiftsvt: 

RT @BamBamland502: 210412 [Facebook] BamBam Play The Sims update

BamBam will be live on Facebook streaming The Sims 4!

⏰ Apr 15 at 10PM K…
Long: None, Lati: None
RT @MikeCarlton01: When two prime ministers - one Labor, one Liberal - denounce the Murdochracy attack on our democracy, you begin to think…
Long: None, Lati: None
RT @7makemoney: Facebook Ads University_Review

Learn More: https://t.co/C6GanXKy1L

Facebook Ads are currently one of the best and most po…
Long: None, Lati: None
RT @radian_iu: 210412 Samdasoo IG Update with #아이유 #IU 

🔗https://t.co/lS5ci0Q8hh
🔗https://t.co/FmE1JrrtpN https://t.co/Vzp0t77kkc
Long: None, Lati: None
Paalam YOUTUBE :(

Here's the link to my new channel.
Channel: Roland Monilla
https://t.co/qvBw401j8c...​

Connect… https://t.co/5n0WnzzTHb
Long: None, Lati: None
RT @shayndenniz: happy 1.4 million followers on facebook! :&gt; tysm ily all ♡ https://t.co/N3kJfK9Xek
Long: None, Lati: None
RT @SusanSmithAus: @JoshButler Also Scott Morrison

"I've been ve

Read more here: https://t.co/…
Long: None, Lati: None
RT @liittlebirdd: Mahesh babu all areas - B A RAJU
Long: None, Lati: None
RT @ygent_official: #강승윤(KANG SEUNG YOON) 1st FULL ALBUM [PAGE] 
Online Fan signing event
Notice has been uploaded. 

▶️https://t.co/CHREBr…
Long: None, Lati: None
RT @ygent_official: #강승윤(KANG SEUNG YOON) 1st FULL ALBUM [PAGE] 
Online Fan signing event
Notice has been uploaded. 

▶️https://t.co/CHREBr…
Long: None, Lati: None
RT @kyawhsanhlaing1: The military is now charging families 120,000 Myanmar kyat ($85) to retrieve the bodies of relatives who died Friday,…
Long: None, Lati: None
Brain as a memory chip to control

1. 🧔🏾 installed memory chip in 📱💻 devices. 

2. 🧔🏾 installed memory chip in Robo… https://t.co/zANEDRpeEj
Long: None, Lati: None
RT @AmorDeViceGanda: NAGSTART NA!!!! Watch na!!!
🔵 CLICK THE LINK BELOW:

https://t.co/43I8AzxHt6

QueenVGxSaridon FB LIVE
Long: None, Lati: None
RT @FBoversight: This is beyond shocking. A deep dive into Facebook's i

RT @NYCDEATHROWCATS: Jack is a handsome neutered boy who was dumped by his owner at the shelter. Jack is hoping to meet a new furever frien…
Long: None, Lati: None
RT @kyawhsanhlaing1: The military is now charging families 120,000 Myanmar kyat ($85) to retrieve the bodies of relatives who died Friday,…
Long: None, Lati: None
RT @RajuBistaBJP: Hon'ble HM Shri @AmitShah ji's roadshow in Kalimpong!

#Ebar200Paar #EsPaliBJP

Watch Live :  https://t.co/fuO3M4BKq2
Long: None, Lati: None
PSA:  PLEASE READ 
ON MY FACEBOOK PAGE
https://t.co/B2PhuMXvPE             Water Flag Awareness designs property of… https://t.co/KWr6y1WWj9
Long: None, Lati: None
Contact for appointment:
+92-321-5040001
+92-51-2609277
Visit our website: https://t.co/y81IdJYosb
YouTube Channel:… https://t.co/kB0Oh2NMZh
Long: None, Lati: None
RT @lingerinmeadow: Let's drop by and scroll through their other social media platforms to ❤️/👍 today's Haram 퉤하람 Youngjae contents as well…
Long: None, Lati: None
RT @43a6f0ce5dac4ea: @

RT @FaustoCarvajal: Facebook and Instagram are down https://t.co/pgwgItgi5r #health #business #engage
Long: None, Lati: None
@vionaserrano @toastedmorena paki ❤️ po please thank you po❤️

https://t.co/BHqYIUh7od
Long: None, Lati: None
RT @WORLDMUSICAWARD: Happy Birthday to #EXO's gorgeous and multitalented #SEHUN! 👨‍🎤🎸🎂🍾🎁🎉🎇☀️👑 #HappySehunDay #28thSpringWithSEHUN  #세훈 @wea…
Long: None, Lati: None
RT @kyawhsanhlaing1: The military is now charging families 120,000 Myanmar kyat ($85) to retrieve the bodies of relatives who died Friday,…
Long: None, Lati: None
RT @GrogsGamut: Beat appearing before journalists who might ask questions
Long: None, Lati: None
https://t.co/4qF0I4daGS Sto lat
Long: None, Lati: None
Ay... what is this? 😭
Long: None, Lati: None
@S70729338 @Facebook I recommended you all to @supreme_cr4ck he's good an legit got mine done in sec
Long: None, Lati: None
Just deeply frustrating. It’s not like all those spreading this hatred are behind elaborately disguised accounts. A… 